### はじめに

<div align="center">
    <img alt="Logo" src="https://hikettei.github.io/cl-waffe-docs/cl-waffe.png" width="45%">
</div>

cl-waffe2はANSI Common Lisp上に、深層学習モデルの設計や学習に特化した行列演算ライブラリとそれに関連するパッケージを提供するフレームワークです。このNotebookでは複数のセクションに分けてcl-waffe2の基本的な使い方をサンプルコードと共に提供します。

cl-waffe2のインストール方法やドキュメントについては、[公式ドキュメント](https://hikettei.github.io/cl-waffe2/)をご活用ください。

### common-lisp-jupyter

Jupyter Labでこのファイルをご覧になっている場合、付属しているセルを実行するためには、[common-lisp-jupyter](https://github.com/yitzchak/common-lisp-jupyter)という拡張ライブラリを予め導入していただく必要があります。（インストール方法に関しては、ライブラリのReadme.mdファイル及び公式ドキュメントをご覧ください）

### cl-waffe2を読み込む

cl-waffe2は執筆時点(2023/08/27)では開発途中で、まだQuicklispに登録がされていません。そのため動作するには[オリジナルのGithubリポジトリ](https://github.com/hikettei/cl-waffe2.git)からリポジトリをクローンしていただいたのち、`cl-waffe2.asd`ファイルを読み込んでいただく必要があります。


```sh
$ git clone https://github.com/hikettei/cl-waffe2.git
$ cd cl-waffe2
$ jupyter lab
```

In [1]:
(load "../../../cl-waffe2.asd") ;; 相対パスが読み込めない場合適度調節してください

(asdf:load-system :cl-waffe2 :silent t)

T

T

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining PERFORM (#<STANDARD-CLASS ASDF/LISP-ACTION:TEST-OP> #<SB-MOP:EQL-SPECIALIZER #<SYSTEM "cl-waffe2/test">>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining LPARALLEL.KERNEL:KERNEL-NAME in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFGENERIC: redefining CL-WAFFE2/VM.GENERIC-TENSOR:CURRENT-BACKEND-STATE in DEFGENERIC
SB-KERNEL:REDEFINITION-WITH-DEFGENERIC: redefining CL-WAFFE2/VM.NODES:FORWARD in DEFGENERIC
SB-KERNEL:REDEFINITION-WITH-DEFGENERIC: redefining CL-WAFFE2/VM.NODES:BACKWARD in DEFGENERIC
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CL-WAFFE2/VM.GENERIC-TENSOR:MOVETENSOR-P in DEFUN


## パッケージ管理

まず`section-0-basic`パッケージを定義し、そこでAPIを試すことにしましょう。

cl-waffe2は提供する機能ごとにパッケージの名前空間を分離させています。

### 中心システム

| package | description |
| ------- | ----------- |
| :cl-waffe2/vm | cl-waffe2が動作する仮想マシンです。cl-waffe2 IRに関するAPIなど |
| :cl-waffe2/vm.nodes| AbstractNodeに関する拡張的な機能を提供します。 |
| :cl-waffe2/vm.generic-tensor | AbstractTensorに関する拡張的な機能を提供します。 |

### API

| package | description |
| ------- | ----------- |
| :cl-waffe2 | ネットワーク定義や設定に関するユーティリティを提供します |
| :cl-waffe2/base-impl | !addや!reshapeなど基本的なAPIを提供します |
| :cl-waffe2/distributions | randn関数など、テンソルを確率分布から初期化するためのAPIを提供しています |
| :cl-waffe2/nn | 回帰モデルやCNN, 誤差関数など基本的なネットワークを提供します |
| :cl-waffe2/optimizers | AdamやSGDなど最適化関数の実装を提供します |

### バックエンドの一覧

| package | description |
| ------- | ----------- |
| :cl-waffe2/backends.lisp | ANSI Common Lisp上で実装されたバックエンドです。 |
| :cl-waffe2/backends.cpu | OpenBLASやSIMD Extensionなど、外部ライブラリの力を借りて高速化するバックエンドです。|
| :cl-waffe2/backends.jit.cpu | (試験的) cl-waffe2のコードをCにJITコンパイルすることで動作するバックエンドです。 |
| :cl-waffe2/backends.jit.lisp | (廃止予定) cl-waffe2のコードをLispにJITコンパイルすることで動作するバックエンドです。 |

In [2]:
(defpackage :section-0-basic
    (:use
     :cl
     :cl-waffe2
     :cl-waffe2/base-impl
     :cl-waffe2/vm
     :cl-waffe2/vm.nodes
     :cl-waffe2/vm.generic-tensor
     :cl-waffe2/distributions))

(in-package :section-0-basic)

#<PACKAGE "SECTION-0-BASIC">

#<PACKAGE "SECTION-0-BASIC">

## 基本的なデータ型

数学では, 一つの数値をスカラー, 一次元の行列をVector, 二次元の行列をMatrixのように表現します。

$$ 1 $$

$$ (a_1,a_2,\dots,a_n) $$

\begin{pmatrix}
1 & 2 \\
3 & 4 \\
\end{pmatrix}

便宜上、cl-waffe2ではこれら全てのデータ型を内包する`AbstractTensor`というデータ型を用いて計算を進めていきます。`:cl-waffe2/vm.generic-tensor`パッケージが提供する[make-tensor](https://hikettei.github.io/cl-waffe2/generic-tensor/#function-make-tensor)関数を用いることで新しいTensorを作成することができます。

更に、[:cl-waffe2/distributions](https://hikettei.github.io/cl-waffe2/distributions/)パッケージは確率分布からテンソルをサンプリングする様々な関数を提供しています。

In [3]:
(make-tensor 1)

{SCALARTENSOR[float]   
    1.0
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [4]:
(make-tensor '(10 10) :initial-element 1.0)

{CPUTENSOR[float] :shape (10 10)  
  ((1.0 1.0 1.0 ~ 1.0 1.0 1.0)           
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0)   
        ...
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0)
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [5]:
(randn `(10 10)) ;; ガウス分布からサンプリング

{CPUTENSOR[float] :shape (10 10)  
  ((0.1371898    0.98653716   -0.41064423  ~ -2.1985002   0.64959      0.37606597)                    
   (-0.990578    1.8650554    1.549668     ~ 2.007281     0.2079959    1.2413626)   
                 ...
   (-0.14403512  -0.13514197  -0.2724243   ~ 0.59860396   0.87737995   -0.39738473)
   (0.3234897    1.256502     -0.66197085  ~ 1.356871     0.502143     0.6284781))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [6]:
;; Storageにアクセスする
(tensor-vec (make-tensor 1))

1.0

In [7]:
(make-input `(2 2) nil)
(tensor-vec (make-input `(2 2) nil)) ;; tensor-vecを呼び出して初めて割り当てられる

{CPUTENSOR[float] :shape (2 2) :named ChainTMP605 
  <<Not-Embodied (2 2) Tensor>>
  :facet :input
  :requires-grad NIL
  :backward NIL}

#(0.0 0.0 0.0 0.0)

## AbstractTensor

<div align="center">
    <img alt="Logo" src="../assets/AbstractTensor.png" width="45%">
</div>

`storage`に相当するデータ型(例: `fixnum`や`Common Lisp標準配列`など...)を`AbstractTensor`でラップすることで、更に以下の情報を付与することができます。

- `:requires-grad` 逆伝播をしたのちに、勾配を求めるかどうか

- `:backward` 計算ノードの情報

- `:facet` テンソルのメモリ割り当ての状態 `:exist`であれば存在する `:input`であれば未割り当て

etc...

また、`AbstractTensor`と`storage`のデータ型は`change-facet`関数を用いて強制的に行うことができます。また、[convert-tensor-facet](https://hikettei.github.io/cl-waffe2/utils/#generic-convert-tensor-facet)メソッドを追加することで任意の組み合わせの変換を拡張することが可能です。

```lisp
;; 使い方
(change-facet 対象 :direction 変換先のデータ型)
```

In [8]:
(change-facet #2A((1 2 3)
		          (4 5 6)
		          (7 8 9))
	       :direction 'AbstractTensor)

{CPUTENSOR[int32] :shape (3 3)  
  ((1 2 3)
   (4 5 6)
   (7 8 9))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [9]:
(change-facet (ax+b `(3 3) 0 1) :direction 'array)

#2A((1.0 1.0 1.0) (1.0 1.0 1.0) (1.0 1.0 1.0))

In [10]:
;; change-facet前後でテンソルのポインタは共有されています
;; そのため、AbstractTensorを一旦CL配列にして編集した後、AbstractTensorに戻すといった挙動も可能です。
;; with-facetマクロはchange-facet関数を呼び出した後、自動でa*に結果をbindingしてくれます。

;; 1で埋め尽くされた3x3行列の対角を0.0で埋める
(let ((a (ax+b `(3 3) 0 1)))
  (with-facet (a* (a :direction 'array))
    (setf (aref a* 0 0) 0.0)
    (setf (aref a* 1 1) 0.0)
    (setf (aref a* 2 2) 0.0))
   a)

{CPUTENSOR[float] :shape (3 3)  
  ((0.0 1.0 1.0)
   (1.0 0.0 1.0)
   (1.0 1.0 0.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

## 演算をする

`AbstractTensor`を組み合わせて計算を行いましょう。`!add`関数は与えられた二つのTensorの要素ごとの和を求める関数です。しかし`!add`を呼び出しただけでは何も起こりません。

In [11]:
(!add 1 1)

{SCALARTENSOR[int32]  :named ChainTMP693 
  :vec-state [maybe-not-computed]
  <<Not-Embodied (1) Tensor>>
  :facet :input
  :requires-grad NIL
  :backward <Node: SCALARANDSCALARADD-SCALARTENSOR (A[SCAL] B[SCAL] -> A[SCAL]
                                                    WHERE SCAL = 1)>}

`:backward`には`ScalarAndScalarAdd-ScalarTensor`という計算ノードが記録されています。そして`:facet`は`:input`になっていて、結果を格納する行列の割り当ては行われていません。

`cl-waffe2`は演算を呼び出してもその時点では実行されず、ある地点でコンパイルが実行されて初めて演算が呼び出されます。

命令をコンパイルするためには二つの関数が用意されています。`proceed`と`build`です。

### proceed

proceed系統の関数はどれもREPL上で、即実行を目指した実行方式を選択します。具体的には、計算木を木構造のまま処理し実行します（インタプリタ的に）。これによってコンパイル時間によるオーバーヘッドを削減します。

他にも:

`proceed-backward` 与えれらたテンソルを実行したのち、逆伝播も計算し、パラメーターの勾配を求めます。

`proceed-time` インタプリタのまま実行したテンソルの実行時間を二回に分けて計測します。

`proceed-bench` 計算ノードをコンパイルした後、ベンチマークを実行します。（インタプリタでは動作しません）

In [12]:
;; Proceed
(proceed (!add 1 1))

{SCALARTENSOR[int32]  :named ChainTMP712 
  :vec-state [computed]
    2
  :facet :input
  :requires-grad NIL
  :backward <Node: PROCEEDNODE-T (A[~] -> A[~])>}

In [13]:
;; Proceed-backward
(let ((x (parameter (ax+b `(3 3) 0 2))))
     (proceed-backward
      (!sum
       (!mul x 3.0)))
     (grad x))

{CPUTENSOR[float] :shape (3 3)  
  ((3.0 3.0 3.0)
   (3.0 3.0 3.0)
   (3.0 3.0 3.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}

In [14]:
;; Proceed-time
(proceed-time (!mul 2.0 2.0))

{SCALARTENSOR[float]  :named ChainTMP1171 
  :vec-state [computed]
    4.0
  :facet :input
  :requires-grad NIL
  :backward <Node: PROCEEDNODE-T (A[~] -> A[~])>}

Proceed-Time: With allocation time:
Evaluation took:
  0.022 seconds of real time
  0.007703 seconds of total run time (0.007543 user, 0.000160 system)
  36.36% CPU
  27 lambdas converted
  51,403,628 processor cycles
  3,756,544 bytes consed
  
Proceed-Time: Without allocation time:
Evaluation took:
  0.000 seconds of real time
  0.000017 seconds of total run time (0.000017 user, 0.000000 system)
  100.00% CPU
  36,098 processor cycles
  0 bytes consed
  


In [15]:
;; proceed-bench
;; *が付いている命令 -> 実行時間が平均以上
;; 実行時間が大きいノードtopKをソートして表示する

(proceed-bench (!sum (ax+b `(10 10) 0 1)) :n-sample 10000)

{CPUTENSOR[float] :shape (1 1) -> :view (<(BROADCAST 1)> <(BROADCAST 1)>) -> :visible-shape (1 1) :named ChainTMP1193 
  ((100.0))
  :facet :input
  :requires-grad NIL
  :backward NIL}

[Sorted by Instructions]
 Time(s)   |   Instruction ( * - Beyonds the average execution time)
0.010573   | <WfInst[Compiled: SCALARMUL-CPUTENSOR] : TID1194 <= op(TID1194(1 1) <Input>TID1196(1))>
0.005035   | <WfInst[Compiled: VIEWTENSORNODE-T]    : TID1205 <= op(TID1205(10 10) TID1194(1 1))>
0.057372*  | <WfInst[Compiled: ADDNODE-CPUTENSOR]   : TID1205 <= op(TID1205(10 10) <Input>TID1191(10 10))>
0.005339   | <WfInst[Compiled: VIEWTENSORNODE-T]    : TID1227 <= op(TID1227(1 1) TID1205(10 10))>

4 Instructions | 5 Tensors

 Total Time: 0.078319 sec

[Sorted by topK]
 Instruction                           | Total time (s) | Time/Total (n-sample=10000)
<WfInst[Compiled: ADDNODE-CPUTENSOR]   | 0.057372       | 73.25426%
<WfInst[Compiled: SCALARMUL-CPUTENSOR] | 0.010573       | 13.499917%
<WfInst[Compiled: VIEWTENSORNODE-T]    | 0.010374       | 13.245829%


`proceed`系統の関数を用いてREPL上などでデバッグしながらボトルネックを探しつつ、計算ノードを組み立てることが可能です。

ある程度ネットワークが出来上がってきて、学習や推論のフェーズに移行するときは`build`関数を用います。これは、`proceed-bench`が表示したものと同じ形式の命令列に計算ノードをコンパイルして実行するモードです。多少のコンパイル時間と引き換えに最も高速に動作します。

In [16]:
(let ((x (parameter (ax+b `(10 10) 0 1)))
      (y (parameter (ax+b `(10 10) 0 3))))
     (let ((compiled-model (build (!sum (!mul x y)))))
          (format t "[Forward]: ~%~a~%" (forward compiled-model))
          (backward compiled-model)
          (format t "~%[X.grad]:~%~a~%[Y.grad]:~%~a~%" (grad x) (grad y))
          nil))

NIL

[Forward]: 
{CPUTENSOR[float] :shape (1 1) -> :view (<(BROADCAST 1)> <(BROADCAST 1)>) -> :visible-shape (1 1) :named ChainTMP1310 
  ((300.0))
  :facet :input
  :requires-grad NIL
  :backward NIL}

[X.grad]:
{CPUTENSOR[float] :shape (10 10)  
  ((3.0 3.0 3.0 ~ 3.0 3.0 3.0)           
   (3.0 3.0 3.0 ~ 3.0 3.0 3.0)   
        ...
   (3.0 3.0 3.0 ~ 3.0 3.0 3.0)
   (3.0 3.0 3.0 ~ 3.0 3.0 3.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}
[Y.grad]:
{CPUTENSOR[float] :shape (10 10)  
  ((1.0 1.0 1.0 ~ 1.0 1.0 1.0)           
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0)   
        ...
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0)
   (1.0 1.0 1.0 ~ 1.0 1.0 1.0))
  :facet :exist
  :requires-grad NIL
  :backward NIL}


## デバイスを切り替える

```lisp
(with-devices (&rest devices)
    body)
```

cl-waffe2の演算(AbstractNode)を実行するデバイスなどは、ユーザーによって拡張したり変更することが簡単になるように設計されています。

利用可能なバックエンド一覧やその状態などは`(show-backends)`を用いて確認することができます。

In [17]:
(show-backends)

NIL


─────[All Backends Tree]──────────────────────────────────────────────────

[*]CPUTENSOR: OpenBLAS=available *simd-extension-p*=available
    └[-]JITCPUTENSOR: compiler=gcc flags=(-fPIC -O3 -march=native) viz=NIL

[*]LISPTENSOR: Common Lisp implementation on matrix operations
    └[-]JITLISPTENSOR: To be deleted in the future release. do not use this.

[-]SCALARTENSOR: is a special tensor for representing scalar values.
    └[-]JITCPUSCALARTENSOR: Use with JITCPUTensor

([*] : in use, [-] : not in use.)
Add a current-backend-state method to display the status.
─────[*using-backend*]───────────────────────────────────────────────────

Priority: Higher <───────────────────>Lower
                  CPUTENSOR LISPTENSOR 

(use with-devices macro or set-devices-toplevel function to change this parameter.)


上のグラフが木構造になっているのは継承関係を表しているからです。

例えば`CPUTensor`を継承してある演算の最適化に特化した`MyTensor`を定義することができます。ここで、`MyTensor`と`CPUTensor`は継承関係にあるので、`MyTensor`に定義されていない演算は`CPUTensor`のものを代わりに用いることが可能です。ですから下のコードは動作します。

ここで`MyTensor`に演算を追加したり再定義させることも可能ですが、別のセクションで触れます。

In [18]:
(defclass MyTensor (cl-waffe2/backends.cpu:CPUTensor) nil)

(with-devices (MyTensor cl-waffe2/backends.cpu:CPUTensor)
    (proceed
     (!add (ax+b `(3 3) 1 0) (ax+b `(3 3) 1 0))))

#<STANDARD-CLASS SECTION-0-BASIC::MYTENSOR>

{MYTENSOR[float] :shape (3 3) :named ChainTMP2203 
  :vec-state [computed]
  ((0.0  2.0  4.0)
   (6.0  8.0  10.0)
   (12.0 14.0 16.0))
  :facet :input
  :requires-grad NIL
  :backward <Node: PROCEEDNODE-T (A[~] -> A[~])>}

## 高次への拡張

Theanoなどのフレームワークと同様に、かつNumpyなどのライブラリとは対称的に、Broadcastingは宣言された軸でないと自動で適用されません。

### Rank Upのルール

NumpyのBroadcastingのルールの一つ目は

  - 演算されるテンソル同士のランクが異なる場合、小さい方は大きい方に合わせて先頭に1を足す

というものです。これに対応する演算が`!flexible`という関数になります。これは`<1 x N>`という軸を`:at`で指定された位置に追加します。

In [19]:
(!flexible (ax+b `(3 3) 0 1) :at 0)
(!flexible (ax+b `(3 3) 0 1) :at 1)
(!flexible (ax+b `(3 3) 0 1) :at 2)

;; or

(%transform (ax+b `(3 3) 0 1)[i j] -> [i ~ j])

{CPUTENSOR[float] :shape (<1 x N> 3 3) :named TENSOR 
  :vec-state [maybe-not-computed]
  ((1.0 1.0 1.0)
   (1.0 1.0 1.0)
   (1.0 1.0 1.0))
  :facet :input
  :requires-grad NIL
  :backward <Node: FLEXIBLE-RANK-NODE-T (A[~] -> A[~])>}

{CPUTENSOR[float] :shape (3 <1 x N> 3) :named TENSOR 
  :vec-state [maybe-not-computed]
  ((1.0 1.0 1.0)
   (1.0 1.0 1.0)
   (1.0 1.0 1.0))
  :facet :input
  :requires-grad NIL
  :backward <Node: FLEXIBLE-RANK-NODE-T (A[~] -> A[~])>}

{CPUTENSOR[float] :shape (3 3 <1 x N>) :named TENSOR 
  :vec-state [maybe-not-computed]
  ((1.0 1.0 1.0)
   (1.0 1.0 1.0)
   (1.0 1.0 1.0))
  :facet :input
  :requires-grad NIL
  :backward <Node: FLEXIBLE-RANK-NODE-T (A[~] -> A[~])>}

{CPUTENSOR[float] :shape (3 <1 x N> 3) :named TENSOR 
  :vec-state [maybe-not-computed]
  ((1.0 1.0 1.0)
   (1.0 1.0 1.0)
   (1.0 1.0 1.0))
  :facet :input
  :requires-grad NIL
  :backward <Node: FLEXIBLE-RANK-NODE-T (A[~] -> A[~])>}

In [22]:
(let ((a (ax+b `(3 1) 1 0))
      (b (ax+b `(3 3) 0 0)))
     (proceed (!add b (!view a (broadcast-to b)))))

{CPUTENSOR[float] :shape (3 3) :named ChainTMP2550 
  :vec-state [computed]
  ((0.0 0.0 0.0)
   (1.0 1.0 1.0)
   (2.0 2.0 2.0))
  :facet :input
  :requires-grad NIL
  :backward <Node: PROCEEDNODE-T (A[~] -> A[~])>}

## defmodel

`defmodel`というマクロは、複数の`AbstractNode`を束ねた`Composite`(他のライブラリではModuleやModelに相当)というCLOSクラスを定義します。加えて`Composite`はその他に、静的に動作する関数を定義するなどにも応用が可能です。

## defsequence

```lisp
(call (XXX)
      (call (YYY)
            (call (ZZZ) X)))
```

Instead,

```lisp
(call-> X
        (XXX)
        (YYY)
        (ZZZ))
```

`call->` ... 複数の計算ノードを合成する

## deftrainer + MLP学習 + 最適化

=> `1device-extension.ipynb`へ続く・・・